In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor
import subprocess
import time
import re
import pandas as pd
import numpy as np
import PyPDF2
import os
import ast
from tqdm import tqdm
from collections import OrderedDict
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import subprocess
import time
import undetected_chromedriver as uc
import re

In [2]:
def parse_paper_list(file_path):
    result_dict = {}
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in lines:
        parts = [p.strip() for p in line.strip().split("|")]
        if len(parts) == 3:
            scopus_id, title, link = parts
            combined_title = f"{title}, {title}"
            result_dict[combined_title] = {
                "scopus_id": scopus_id,
                "link": link
            }

    return result_dict


In [3]:
all_pdfs = parse_paper_list("./Downloads/failed_pdfs_10Jul.txt")
all_pdfs

{'Boosting the Supercapacitive Performance via Incorporation of Vanadium in Nickel Phosphide Nanoflakes A HighPerformance Flexible Renewable Energy Storage Device, Boosting the Supercapacitive Performance via Incorporation of Vanadium in Nickel Phosphide Nanoflakes A HighPerformance Flexible Renewable Energy Storage Device': {'scopus_id': '85127315546',
  'link': 'https://pubs.acs.org/doi/pdf/10.1021/acs.energyfuels.2c00315'},
 'Synthetic Methodologies and Energy StorageConversion Applications of Porous Carbon Nanosheets A Systematic Review, Synthetic Methodologies and Energy StorageConversion Applications of Porous Carbon Nanosheets A Systematic Review': {'scopus_id': '85126628296',
  'link': 'https://pubs.acs.org/doi/pdf/10.1021/acs.energyfuels.2c00077'},
 'Fabrication and Performance of ShapeStable Phase Change Composites Supported by EnvironmentFriendly and Economical Loofah Sponge Fibers for Thermal Energy Storage, Fabrication and Performance of ShapeStable Phase Change Composites

In [4]:
len(all_pdfs)

568

In [5]:
def slice_dict(d, start, end):
    return dict(list(d.items())[start:end])

In [6]:
batch = slice_dict(all_pdfs, 0, 3000)
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

remove_df = pd.read_excel('/Users/dtuser/Downloads/Remove_Scopus.xlsx')

# Extract SCOPUS IDs from Excel (assuming the column is named something like 'scopus_id')
remove_scopus_ids = set(remove_df.iloc[:, 1].astype(str).str.extract(r'SCOPUS_ID:(\d+)', expand=False).dropna())


# Check for overlap
overlap_ids = all_scopus_ids_in_data.intersection(remove_scopus_ids)

# Report result
if not overlap_ids:
    print("✅ No SCOPUS IDs in the batch are present in the Excel file.")
else:
    print(f"❌ {len(overlap_ids)} SCOPUS IDs found in both batch and Excel file.")
    print("Conflicting IDs:", overlap_ids)

✅ No SCOPUS IDs in the batch are present in the Excel file.


In [7]:
def open_pdf_in_safari_and_save(pdf_url):
    apple_script = f"""
    tell application "Safari"
    open location "{pdf_url}"
    delay 3 -- Wait for the page to load and for the PDF to render
    end tell
    
    
    tell application "System Events"
        tell process "Safari"
            set frontmost to true  -- Ensure Safari remains the active window
            delay 10
        end tell
    
        -- Click the download button inside Safari
        do shell script "cliclick c:1039,1131"
        delay 4
    end tell
    
    tell application "Safari"
        activate
        tell front window
            close current tab
        end tell
    end tell
    """

    subprocess.run(["osascript", "-e", apple_script], text=True)

In [8]:
def clear_history():
    apple_script = """
    tell application "Safari"
        activate  -- Bring Safari to the front
    end tell

    delay 2  -- Give Safari time to come forward

    tell application "System Events"
        tell process "Safari"
            set frontmost to true  -- Ensure Safari remains the active window
            delay 1
        end tell

        -- Click at position 1
        do shell script "cliclick c:72,0"
        delay 1.5

        -- Click at position 2
        do shell script "cliclick c:129,209"
        delay 1.5

        -- Click at position 3
        do shell script "cliclick c:1110,659"
        do shell script "cliclick c:1110,659"
        do shell script "cliclick c:1102,425"
        do shell script "cliclick c:1102,425"
        do shell script "cliclick c:1088,408"
        do shell script "cliclick c:1088,408"
        delay 3
    end tell
    """

    import subprocess
    subprocess.run(["osascript", "-e", apple_script], text=True)

In [9]:
def new_tab():
    apple_script = """
    tell application "Safari"
        activate
        if not (exists window 1) then
            make new document
        else
            tell window 1
                make new tab
            end tell
        end if
    end tell
    """
    import subprocess
    subprocess.run(["osascript", "-e", apple_script], text=True)

In [10]:
os.mkdir('/Users/dtuser/Downloads/failed_10Jul')

In [13]:
batch = slice_dict(all_pdfs, 0, 3000)

folder_path = '/Users/dtuser/Downloads/failed_10Jul'

scopus_ids_in_folder = set()
for filename in os.listdir(folder_path):
    if '-' in filename:
        try:
            scopus_id_part = filename.rsplit('-', 1)[-1].split('.')[0]
            scopus_ids_in_folder.add(scopus_id_part)
        except IndexError:
            continue
    else:
        try:
            if int(filename[:-4]):
                scopus_id_part = filename[:-4]
                scopus_ids_in_folder.add(scopus_id_part)
        except:
            continue
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

missing_scopus_ids = all_scopus_ids_in_data - scopus_ids_in_folder

missing_entries = {title: info for title, info in batch.items() if info['scopus_id'] in missing_scopus_ids}
len(scopus_ids_in_folder), len(missing_entries)

(554, 14)

In [12]:
download_path = './Downloads/failed_10Jul'
count = 0
# clear_history()
for title, info in missing_entries.items():
    count += 1
    # if count % 25 == 0:
    #     clear_history()
    if count % 30 == 0:
        new_tab()
    link = info['link']
    scopus_id = info['scopus_id']
    try:
        open_pdf_in_safari_and_save(link)
        time.sleep(2)
        files = os.listdir('./Downloads/')
        pdf_files = [f for f in files if f.endswith(".pdf")]
        current_time = time.time()
        recent_files = [
            os.path.join('', f)
            for f in pdf_files
            if current_time - os.path.getctime(os.path.join('./Downloads/', f)) <= 10
        ]
        
        if recent_files == []:
            continue
        safe_title = re.sub(r'</?inf>', '', title)
        safe_title = re.sub(r'[^A-Za-z0-9 ]+', '', safe_title)
        safe_title = safe_title.strip() + '-' + scopus_id
        new_name = os.path.join(download_path, f"{safe_title}.pdf")  # Rename with title
        latest_file = max([os.path.join('./Downloads', f) for f in recent_files], key=os.path.getctime)
        try:
            os.rename(latest_file, new_name)
            print(f"📂 Renamed file to: {new_name}")
        except OSError as e:
            if e.errno == 63:  
                fallback_name = os.path.join(download_path, f"{scopus_id}.pdf")
                os.rename(latest_file, fallback_name)
                print(f"⚠️ Filename too long, saved as: {fallback_name}")
            else:
                raise e  # Reraise other unexpected errors
        continue
    except Exception as e:
        print(f"❌ Error processing ScienceDirect PDF: {e}")
        continue   

⚠️ Filename too long, saved as: ./Downloads/failed_10Jul/85127140926.pdf
⚠️ Filename too long, saved as: ./Downloads/failed_10Jul/85126662334.pdf
📂 Renamed file to: ./Downloads/failed_10Jul/Pneumatic QuasiPassive Variable Stiffness Mechanism for Energy Storage Applications Pneumatic QuasiPassive Variable Stiffness Mechanism for Energy Storage Applications-85122850204.pdf
📂 Renamed file to: ./Downloads/failed_10Jul/Prediction Markets for Probabilistic Forecasting of Renewable Energy Sources Prediction Markets for Probabilistic Forecasting of Renewable Energy Sources-85115176150.pdf
📂 Renamed file to: ./Downloads/failed_10Jul/Perspectives on Future Power System Control Centers for Energy Transition Perspectives on Future Power System Control Centers for Energy Transition-85128609501.pdf
📂 Renamed file to: ./Downloads/failed_10Jul/CECRE Supervision and Control of Spanish Renewable Energies in the Last 15 years CECRE Supervision and Control of Spanish Renewable Energies in the Last 15 year